In [ ]:
import pandas as pd
import numpy as np
import math

from sklearn.model_selection import cross_validate, train_test_split
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE

from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    classification_report,
    ConfusionMatrixDisplay
)

from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import DBSCAN
from pathlib import Path

In [ ]:
data_file = Path(input("Enter path to your data file (.xlsx): ")).expanduser()
new_data = pd.read_excel(
    data_file,
    sheet_name="Directly usable data"
)

In [ ]:
new_data = new_data.dropna(axis =0, subset=['corrected Region'])

In [ ]:
all_regions = new_data['corrected Region'].unique()

In [ ]:
regions_list = new_data['corrected Region'].value_counts()
regions_list_df = regions_list.to_frame()
regions_list_df = regions_list_df[regions_list_df['corrected Region'] < 5]
regions_list_df = regions_list_df.reset_index()

regions_to_remove = regions_list_df['index'].unique()

new_data = new_data[~new_data['corrected Region'].isin(regions_to_remove)]

new_data['corrected Region'].nunique()

In [ ]:
regions = new_data["corrected Region"].unique()

In [ ]:
len(regions)

In [ ]:
def dbscan_10percent_outliers_marker(df,regions,file_name):
    new_df = pd.DataFrame()
    for i in range(len(regions)):
        region_df = df[df['corrected Region'] == regions[i]].copy()
        
        min_num_of_neighbours = math.ceil(len(region_df['206Pb/204Pb']) * (10/100))# calculate 10%
        if (min_num_of_neighbours ==1):
            min_num_of_neighbours = 2
            
        selected_columns = ['206Pb/204Pb', '207Pb/204Pb', '208Pb/204Pb']
        region_df_selected = region_df[selected_columns].copy()  # Create a copy to avoid modifying the original DataFrame
        
        scaler = MinMaxScaler()
        region_df_scaled = scaler.fit_transform(region_df_selected)
        
        region_df_scaled = pd.DataFrame(region_df_scaled, columns=selected_columns)
        
        dbscan = DBSCAN(eps=0.2, min_samples=min_num_of_neighbours).fit(region_df_scaled) #trained on 10% min_samples
        region_df['outliers'] = dbscan.labels_
        region_df['group'] = region_df['outliers'].map(lambda x: regions[i] + ' ' + str(x))
        
        new_df = pd.concat([region_df,new_df],axis=0)
    
    return new_df

        
        #output_dir = '/Users/new/Desktop/Thesis/outputs/Outliers/DBscan_outliers/'
        #file_path = os.path.join(output_dir, file_name)


In [ ]:
df_after_dbscan = dbscan_10percent_outliers_marker(new_data,regions,'File name') # In new df column "outliers" added

In [ ]:
df_after_dbscan.to_excel('enter_your_direction_with_outliers.xlsx', index=False) #save with outliers

# Remove outliers and group regions with less then 20 samples

In [ ]:
df_after_dbscan_no_outliers = df_after_dbscan[df_after_dbscan['outliers']!= -1]

In [ ]:
samples_per_region = df_after_dbscan_no_outliers.groupby('corrected Region')['corrected Region'].transform('count')
regions_with_few_samples = samples_per_region <= 20

# Step 3: Assign 1 to 'group' column for samples from regions with 20 or fewer samples
df_after_dbscan_no_outliers.loc[regions_with_few_samples, 'group'] = df_after_dbscan_no_outliers.loc[regions_with_few_samples, 'corrected Region']

In [ ]:
df_after_dbscan_no_outliers.to_excel('enter_your_direction_no_outliers.xlsx',index=False) #save without outliers

# SMOTE + Classification

In [ ]:
new_clustered = pd.read_excel('enter_your_direction_no_outliers.xlsx')

In [ ]:
regions_arr = [r for r in new_clustered['group'].unique() if pd.notna(r)]

In [ ]:
def encoder(df,region_name):
    
    df['class'] = np.where(df['group']==region_name, 1, 0)
    return df

In [ ]:
def smote_generator(df):#k_neighbors=3 changed to 2
    num_of_ones = df['class'].sum()

# Determine k_neighbors based on the count of "1"
    if num_of_ones < 2:
        # Simply return the original X, y untouched (or you could return None)
        X = df[["206Pb/204Pb","207Pb/204Pb","208Pb/204Pb"]]
        y = df['class']
        return X, y
    
    if num_of_ones == 2:
        k_neighbors = 1
    elif num_of_ones == 3:
        k_neighbors = 2
    elif num_of_ones == 4:
        k_neighbors = 3
    elif num_of_ones == 5:
        k_neighbors = 4
    elif num_of_ones == 6:
        k_neighbors = 5
    elif num_of_ones == 7:
        k_neighbors = 6
    elif num_of_ones == 8:
        k_neighbors = 7
    elif num_of_ones == 9:
        k_neighbors = 8
    elif num_of_ones == 10:
        k_neighbors = 9
    elif num_of_ones > 10:
        k_neighbors = 10
        
    oversample = SMOTE(k_neighbors=k_neighbors)
    X = df[["206Pb/204Pb","207Pb/204Pb","208Pb/204Pb"]]
    y = df['class']
    
    X_resampled, y_resampled = oversample.fit_resample(X, y)
    return X_resampled,y_resampled

In [ ]:
def train_binom_class(df,region_name):
    
    encoded_df = encoder(df,region_name)
    X,y = smote_generator(encoded_df)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

    # Skip if only one class present
    if y_train.nunique() < 2:
        print(f"Skipping region {region_name!r}: only one class in training data")
        return None

    # 4) Train & save
    model = XGBClassifier()
    model.fit(X_train, y_train)
    
    model.save_model(f"{region_name}_xgb_model.json")
    return model

In [ ]:
def validate(X,y):
    scoring = ["accuracy","precision","recall","f1"]
    xgb = XGBClassifier()
    
    results = cross_validate(xgb,X,y,cv=10,scoring=scoring)
    
    results['fit_time'] = np.mean(results['fit_time'])
    results['score_time'] = np.mean(results['score_time'])
    results['test_accuracy'] = np.mean(results['test_accuracy'])
    results['test_precision'] = np.mean(results['test_precision'])
    results['test_recall'] = np.mean(results['test_recall'])
    results['test_f1'] = np.mean(results['test_f1'])
    
    
    
    
    return results
    

In [ ]:
def cross_validate_class(df,region_name):
    
    encoded_df = encoder(df,region_name)
    X,y = smote_generator(encoded_df)
    results = validate(X,y)
    #model = train_class(X,y)
    return results

In [ ]:
#FOR CLUSTERS
models = {}

for cluster in regions_arr:
    model = train_binom_class(new_clustered,cluster)
    models[cluster] = model
    if model is not None:
        models[cluster] = model

# Predict new data

In [ ]:
test_data = pd.read_excel("your_data_to_predict.xlsx")
# The data should be organised in an Excel file with column headings: '206Pb/204Pb', '207Pb/204Pb', '208Pb/204Pb'.

In [ ]:
# The prediction function processes one sample at a time.
# To predict the first row, use test_data.head(1).
# To predict the second row, use test_data.iloc[1:2].
# For the third row, use test_data.iloc[2:3], and so on.

probabilities = {}

for clf in models:
    model = models[clf]
    proba = model.predict_proba(test_data.head(1)) #for 2nd line use test_data.iloc[1:2]
    probabilities[clf] = proba
    



In [ ]:
data = []

for region_name, values in probabilities.items():
    class_1 = values[0][1]    
    class_0 = values[0][0]

    data.append({
        'region_name': region_name,
        'class_0': class_0,
        'class_1': class_1,
        'type': 'float32'
    })

df_all = pd.DataFrame(data)

In [ ]:
df_all = df_all.round(3)
df_all.sort_values("class_1",ascending = False)

#The results show the probabilities for a single test sample, sorted from highest to lowest.